In [ ]:
from astropy.io import fits
from astropy.wcs import WCS
import numpy as np
from astropy.coordinates import SkyCoord
import astropy.units as u

# FITS dosyasını aç
fits_file_path = 'Uzak_Pc/Halpha_map.fits'
hdul = fits.open(fits_file_path)

# WCS bilgilerini al
wcs = WCS(hdul[0].header)

# RA ve DEC değerlerini içeren TXT dosyasını oku
txt_file_path = 'Uzak_Pc/Ha_olan_Ra_Dec.txt'  # Gerçek dosya adını ve yolunu belirtin

# Genfromtxt fonksiyonunu kullanarak veriyi yükle
ra_dec_data = np.genfromtxt(txt_file_path, dtype=str, delimiter='\t')

# Her bir RA ve DEC çifti için piksel konumlarını ve değerlerini bul
pixel_values = []
for ra_str, dec_str in ra_dec_data:
    # H:M:S veya D:M:S formatındaki dizileri SkyCoord kullanarak dönüştür
    coord = SkyCoord(ra=ra_str, dec=dec_str, unit=(u.hourangle, u.deg), frame='icrs')
    # Dönüştürülmüş koordinatlardan RA ve DEC değerlerini al
    ra = coord.ra.deg
    dec = coord.dec.deg
    # Piksel konumunu ve değerini hesapla
    pixel_position = wcs.all_world2pix(ra, dec, 0)
    pixel_value = hdul[0].data[int(pixel_position[1]), int(pixel_position[0])]
    pixel_values.append((ra, dec, pixel_position[0], pixel_position[1], pixel_value))

# Çıktıyı bir dosyaya kaydet
output_file_path = 'Uzak_Pc/pixel_values_output.txt'

with open(output_file_path, 'w') as f:
    f.write("RA\tDEC\tPixel_X\tPixel_Y\tPixel_Value\n")
    for i, (ra, dec, pixel_x, pixel_y, pixel_value) in enumerate(pixel_values):
        line = f"{ra}\t{dec}\t{pixel_x}\t{pixel_y}\t{pixel_value}\n"
        f.write(line)

# FITS dosyasını kapat
hdul.close()